# Computing Momentum Flow Lines

This notebook demonstrates how to compute and visualize momentum flow lines from a stress-energy tensor. Momentum flow lines trace the paths of energy and momentum flux through spacetime, providing insight into how energy propagates in warp drive geometries.

The momentum flow lines follow the momentum density vector field $T^{0i}$ (the spatial components of the energy flux), where:
- $T^{01}$ represents momentum flow in the x-direction
- $T^{02}$ represents momentum flow in the y-direction  
- $T^{03}$ represents momentum flow in the z-direction

These flow lines help visualize how the exotic matter required for a warp drive distributes and flows through the spacetime.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import WarpFactory modules
from warpfactory.metrics.alcubierre import get_alcubierre_metric
from warpfactory.solver.energy import get_energy_tensor
from warpfactory.analyzer.momentum_flow import get_momentum_flow_lines

## Get Metric and Energy Tensor

First, we create an Alcubierre warp drive metric and compute its stress-energy tensor. The stress-energy tensor describes the distribution of matter and energy required to generate the warp bubble geometry.

In [ ]:
# Define Alcubierre metric parameters
grid_size = [1, 20, 20, 20]  # [time, x, y, z]
world_center = [(gs + 1) / 2 for gs in grid_size]  # Center of the grid
velocity = 0.5  # Velocity in units of c
radius = 5.0  # Radius of the warp bubble (R)
sigma = 0.5  # Width of the transition region

print(f"Creating Alcubierre metric with:")
print(f"  Grid size: {grid_size}")
print(f"  World center: {world_center}")
print(f"  Velocity: {velocity}c")
print(f"  Bubble radius (R): {radius}")
print(f"  Sigma: {sigma}")

# Create the Alcubierre metric
metric = get_alcubierre_metric(
    grid_size=grid_size,
    world_center=world_center,
    velocity=velocity,
    radius=radius,
    sigma=sigma
)

print(f"\nMetric created: {metric.name}")
print(f"  Shape: {metric.shape}")
print(f"  Index type: {metric.index}")

In [ ]:
# Compute the stress-energy tensor from the metric
print("Computing stress-energy tensor...")
energy_tensor = get_energy_tensor(metric)

print(f"\nEnergy tensor computed: {energy_tensor.tensor_type}")
print(f"  Index type: {energy_tensor.index}")
print(f"  Shape: {energy_tensor.shape}")

## Get Momentum Flow Lines

Now we'll trace momentum flow lines through the spacetime. We start from a grid of initial points and integrate along the momentum density vector field to see how energy flows through the warp bubble region.

The flow lines are computed by:
1. Starting from a grid of initial positions
2. At each position, evaluating the momentum density vector $(T^{01}, T^{02}, T^{03})$
3. Taking a step in the direction of the momentum vector
4. Repeating until the line exits the grid or reaches the maximum number of steps

In [ ]:
# Define the parameters for momentum flow line calculation
ngrid_steps = 4  # Grid spacing for starting points
step_size = 0.75  # Step size for integration
max_steps = 10000  # Maximum steps to trace each line

# Calculate scale factor from energy tensor
# This normalizes the momentum field for better visualization
scale_factor = 1.0 / np.max(np.abs(energy_tensor[(0, 1)]))

print(f"Momentum flow line parameters:")
print(f"  Grid spacing: {ngrid_steps}")
print(f"  Step size: {step_size}")
print(f"  Max steps: {max_steps}")
print(f"  Scale factor: {scale_factor:.6e}")

# Build a grid of starting points using meshgrid
# Create points at regular intervals across the spatial grid
x_coords = np.arange(1, grid_size[1], ngrid_steps)
y_coords = np.arange(1, grid_size[2], ngrid_steps)
z_coords = np.arange(1, grid_size[3], ngrid_steps)

X, Y, Z = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')

# Package starting points as tuple of arrays
start_points = (X, Y, Z)

print(f"\nStarting from {X.size} initial points")
print(f"  X range: {x_coords[0]} to {x_coords[-1]}")
print(f"  Y range: {y_coords[0]} to {y_coords[-1]}")
print(f"  Z range: {z_coords[0]} to {z_coords[-1]}")

In [ ]:
# Find the momentum flow line paths
print("Computing momentum flow lines...")
paths = get_momentum_flow_lines(
    energy_tensor=energy_tensor,
    start_points=start_points,
    step_size=step_size,
    max_steps=max_steps,
    scale_factor=scale_factor
)

print(f"\nComputed {len(paths)} momentum flow lines")
print(f"  Average path length: {np.mean([len(p) for p in paths if len(p) > 0]):.1f} points")
print(f"  Longest path: {max([len(p) for p in paths if len(p) > 0])} points")
print(f"  Shortest path: {min([len(p) for p in paths if len(p) > 0])} points")

## Plot Momentum Flow Lines

Now we'll visualize the momentum flow lines in 3D. We'll plot a segment of each flow line to show the structure of the momentum field around the warp bubble.

The visualization shows:
- **Flow line paths**: Trajectories following the momentum density vector field
- **Flow patterns**: How energy flows into, through, or around the warp bubble region
- **Field structure**: The spatial organization of the momentum density

In [ ]:
# Setup 3D figure
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Define the segment of paths to plot
path_length = 30  # Length of segment to plot
start_idx = 10  # Start index within each path

# Counter for plotted paths
num_plotted = 0

# Plot each momentum flow line
for i, path in enumerate(paths):
    # Only plot if path is long enough
    if len(path) > start_idx + path_length:
        # Extract the segment to plot
        segment = path[start_idx:start_idx + path_length, :]
        
        # Plot the line segment
        ax.plot3D(
            segment[:, 0],  # X coordinates
            segment[:, 1],  # Y coordinates
            segment[:, 2],  # Z coordinates
            'k-',  # Black lines
            linewidth=1.0,
            alpha=0.6
        )
        num_plotted += 1

# Set labels and title
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_zlabel('Z', fontsize=12)
ax.set_title(
    f'Momentum Flow Lines for Alcubierre Warp Drive\n'
    f'({num_plotted} lines shown, velocity={velocity}c, R={radius})',
    fontsize=14,
    fontweight='bold',
    pad=20
)

# Set the camera view angle
ax.view_init(elev=45, azim=45)

# Set aspect ratio to be equal
ax.set_box_aspect([1, 1, 1])

# Add grid for better depth perception
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nVisualization complete!")
print(f"Plotted {num_plotted} flow lines out of {len(paths)} total paths")

## Analyze Flow Line Statistics

Let's examine some statistics about the momentum flow lines to better understand the momentum field structure.

In [ ]:
# Analyze flow line properties
path_lengths = [len(p) for p in paths if len(p) > 0]

print("Momentum Flow Line Statistics:")
print("="*60)
print(f"Total number of paths: {len(paths)}")
print(f"Non-empty paths: {len(path_lengths)}")
print(f"")
print(f"Path Length Statistics:")
print(f"  Mean: {np.mean(path_lengths):.1f} steps")
print(f"  Median: {np.median(path_lengths):.1f} steps")
print(f"  Std Dev: {np.std(path_lengths):.1f} steps")
print(f"  Min: {np.min(path_lengths)} steps")
print(f"  Max: {np.max(path_lengths)} steps")

# Calculate how many paths reached max_steps
paths_at_max = sum([1 for l in path_lengths if l >= max_steps - 1])
print(f"")
print(f"Paths reaching max steps: {paths_at_max} ({100*paths_at_max/len(path_lengths):.1f}%)")

## Multiple Viewing Angles

Let's create additional views of the momentum flow lines from different angles to better understand the 3D structure.

In [ ]:
# Create figure with multiple viewing angles
fig = plt.figure(figsize=(16, 12))

# Define viewing angles: (elevation, azimuth, title)
views = [
    (45, 45, 'Isometric View'),
    (0, 0, 'Side View (YZ plane)'),
    (0, 90, 'Front View (XZ plane)'),
    (90, 0, 'Top View (XY plane)')
]

for idx, (elev, azim, title) in enumerate(views, 1):
    ax = fig.add_subplot(2, 2, idx, projection='3d')
    
    # Plot flow lines
    for path in paths:
        if len(path) > start_idx + path_length:
            segment = path[start_idx:start_idx + path_length, :]
            ax.plot3D(
                segment[:, 0],
                segment[:, 1],
                segment[:, 2],
                'k-',
                linewidth=0.8,
                alpha=0.5
            )
    
    # Set labels and title
    ax.set_xlabel('X', fontsize=10)
    ax.set_ylabel('Y', fontsize=10)
    ax.set_zlabel('Z', fontsize=10)
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Set viewing angle
    ax.view_init(elev=elev, azim=azim)
    
    # Set aspect ratio
    ax.set_box_aspect([1, 1, 1])
    ax.grid(True, alpha=0.3)

plt.suptitle(
    'Momentum Flow Lines - Multiple Viewing Angles',
    fontsize=16,
    fontweight='bold',
    y=0.995
)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Creating a warp drive metric** - We generated an Alcubierre warp drive spacetime with specified parameters

2. **Computing the stress-energy tensor** - Using Einstein's field equations, we derived the matter distribution required for the warp drive

3. **Calculating momentum flow lines** - We traced paths through the momentum density vector field using `get_momentum_flow_lines()`

4. **Visualizing the flow structure** - We created 3D visualizations showing how energy and momentum flow through the warp bubble region

5. **Analyzing flow statistics** - We examined path lengths and other properties to understand the momentum field structure

### Key Results:

- Momentum flow lines reveal the structure of energy flux in the warp drive spacetime
- The flow patterns show how exotic matter and energy are distributed around the warp bubble
- Flow line behavior depends on the warp drive parameters (velocity, radius, sigma)
- The visualization helps identify regions of strong momentum density and energy flow

### Physical Interpretation:

The momentum flow lines trace the Poynting-like vector field of the gravitational stress-energy tensor. In the warp drive:
- **Converging flow lines** indicate regions where energy accumulates
- **Diverging flow lines** indicate regions where energy disperses
- **Flow line density** indicates the magnitude of momentum flux
- **Flow patterns** reveal how the negative energy density required by the warp drive is organized in space

Understanding momentum flow is important for analyzing the stability and feasibility of warp drive designs, as it shows how energy must be distributed and transported to maintain the warp bubble geometry.